In [256]:
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [282]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320



Projet maison n° 1

Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).


In [258]:
geo["CP Ville"] = geo["Code Postal"].str.extract("(.*)") + " " + geo["Commune"].str.extract("(.*)")
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON


In [259]:
import math

In [260]:
def ville(lat, long):
    n = 3 # Nombre de villes les plus proches
    distance = abs(lat-geo["Latitude"])**2 + abs(long-geo["Longitude"])**2
    distance.apply(lambda x : math.sqrt(x))
    ville = geo.loc[distance.nsmallest(n).index, "Commune"].values
    return(ville)

In [281]:
ville(48.8, 2.3)

array(['BAGNEUX', 'CHATILLON', 'FONTENAY-AUX-ROSES'], dtype=object)

In [262]:
# Test de la fonction (prof)
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

array(['BOSJEAN', 'LE TARTRE', 'LE PLANOIS'], dtype=object)


Projet maison n° 2

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

Utiliser la librairie Python json pour parser la colonne "geo_shape".
Donner le décompte des valeurs accédées avec la clé "type". : OK
Donner le décompte des longueurs des listes accédées avec la clé "coordinates". : OK
Quelle commune est la plus complexe géométriquement ? : OK
Quelle commune est la seconde la plus complexe géométriquement ? : OK
Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ? : OK
Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [287]:
import json

In [264]:
# # OLD
# for i in geo["geo_shape"]:
#     dict_json = json.loads(i)
#     if dict_json["type"] not in nb_elements_type :
#         nb_elements_type.append(dict_json["type"])
# len_coord = len(dict_json["coordinates"][0])
        
# print("Types d'éléments : ", nb_elements_type)
# print()
# print("Longeur des listes coordinates : ", len_coord)

In [288]:
# On convertit les données Json
geo["geo_shape"] = geo["geo_shape"].apply(json.loads)

In [289]:
# 1- Comptage du nombre de types uniques
geo["geo_shape"].apply(lambda x : x["type"]).nunique()

2

In [290]:
geo["geo_shape"].apply(lambda x : x["type"]).unique()

array(['Polygon', 'MultiPolygon'], dtype=object)

In [301]:
# Commune la plus complexe :
geo.loc[geo["geo_shape"].apply(lambda x: x["coordinates"]).str.len().nlargest(1).index, "Commune"]


Code INSEE
83069    HYERES
Name: Commune, dtype: object

In [294]:
# 2- Comptage du nombre de couples [long - lat] pour chaque ligne dans geo_shape["coordinates"] --> Création d'une nouvelle colonne
geo["nb_coordinates"] = geo["geo_shape"].apply(lambda x: x["coordinates"]).str.len()
geo

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,nb_coordinates
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{'type': 'Polygon', 'coordinates': [[[4.926273...",46.153426,4.926114,1
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{'type': 'Polygon', 'coordinates': [[[5.430089...",46.009188,5.428017,1
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{'type': 'Polygon', 'coordinates': [[[5.386190...",45.960848,5.372926,1
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{'type': 'Polygon', 'coordinates': [[[4.895580...",45.996180,4.912273,1
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{'type': 'Polygon', 'coordinates': [[[5.614854...",45.749499,5.594320,1
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,3359.0,2.3,"46.0055913782, 5.35760660735","{'type': 'Polygon', 'coordinates': [[[5.413533...",46.005591,5.357607,1
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,518.0,0.7,"45.9367134524, 5.3328092349","{'type': 'Polygon', 'coordinates': [[[5.321986...",45.936713,5.332809,1
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,696.0,0.3,"45.7873565333, 5.65788307924","{'type': 'Polygon', 'coordinates': [[[5.656393...",45.787357,5.657883,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [305]:
# Communes longueur coordinates == 2 & de type Polygon
ville_complexe = geo.loc[geo["geo_shape"].apply(lambda x: x["coordinates"]).str.len()==2, "geo_shape"]

ville_complexe = ville_complexe.loc[geo["geo_shape"].apply(lambda x: x["type"])=="Polygon"]
                                    
geo.loc[ville_complexe.index, "Commune"].values ,  len(ville_complexe.index)
                                    
# ville_complexe.loc["02232"]

(array(['COYOLLES', 'EYGLIERS', 'LA BASTIDE-DE-SEROU', 'PLOUBALAY',
        'SARTENE', 'CALENZANA', 'LANGOGNE', 'SUNDHOUSE', 'EGUISHEIM',
        'BEAUVERNOIS'], dtype=object),
 10)

In [270]:
from shapely.geometry import Polygon

In [271]:
liste_test1 = []
for i in ville_complexe.loc["02232"]["coordinates"]:
    print(i)
#     liste_test1.append((i[0], i[1]))
liste_test1
# UTILISER LA FONCTION CONTAINS ??

[[3.037897560658703, 49.16921270775551], [3.022651486402025, 49.170816855142384], [3.016004215046372, 49.17997922335811], [2.991714837684887, 49.19196418704024], [2.98137479741421, 49.1844740569323], [2.971813659157835, 49.187528930723204], [2.969496165186601, 49.20994111330158], [2.970866610714616, 49.21304305603823], [2.95873860742351, 49.22660554534713], [2.964349025205721, 49.231911431076426], [3.001619992136874, 49.23565651032947], [3.013095594217706, 49.229218996200984], [3.008290195644244, 49.22338489957146], [3.017032136166848, 49.21975199592734], [3.026996333441546, 49.22011876947888], [3.039328911223897, 49.23046290137869], [3.033310889856941, 49.243338284278906], [3.040777749481918, 49.239928826200895], [3.04971736146969, 49.24144476338487], [3.055118307185106, 49.24608118447443], [3.055004734193956, 49.24258418221722], [3.064571289474025, 49.23964873656758], [3.072714287073868, 49.23075304901955], [3.052139968025862, 49.20662530669365], [3.043415415215641, 49.20904712653231

[]